In [1]:
# Install plotly if not already installed
!pip install plotly

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [2]:
from google.colab import files
uploaded = files.upload()


Saving pokemon_data.csv to pokemon_data.csv


In [3]:
df = pd.read_csv("pokemon_data.csv")
df.head()


,id,name,base_experience,height,weight,types,abilities,moves,stats
0,1,bulbasaur,64,7,69,"grass, poison","overgrow, chlorophyll","razor-wind, swords-dance, cut, bind, vine-whip","hp=45, attack=49, defense=49, special-attack=6..."
1,2,ivysaur,142,10,130,"grass, poison","overgrow, chlorophyll","swords-dance, cut, bind, vine-whip, headbutt","hp=60, attack=62, defense=63, special-attack=8..."
2,3,venusaur,236,20,1000,"grass, poison","overgrow, chlorophyll","swords-dance, cut, bind, vine-whip, headbutt","hp=80, attack=82, defense=83, special-attack=1..."
3,4,charmander,62,6,85,fire,"blaze, solar-power","mega-punch, fire-punch, thunder-punch, scratch...","hp=39, attack=52, defense=43, special-attack=6..."
4,5,charmeleon,142,11,190,fire,"blaze, solar-power","mega-punch, fire-punch, thunder-punch, scratch...","hp=58, attack=64, defense=58, special-attack=8..."


In [4]:
# Summary statistics
df.describe(include='all')


,id,name,base_experience,height,weight,types,abilities,moves,stats
count,1302.000000,1302,1302.000000,1302.000000,1302.000000,1302,1302,1268,1302
unique,NaN,1302,NaN,NaN,NaN,221,707,812,1142
top,NaN,terapagos-stellar,NaN,NaN,NaN,normal,levitate,"headbutt, tackle, body-slam, take-down, double...","hp=35, attack=55, defense=40, special-attack=5..."
freq,NaN,1,NaN,NaN,NaN,86,36,21,16
mean,2560.927803,NaN,161.943932,20.467742,980.863287,NaN,NaN,NaN,NaN
std,3949.887748,NaN,82.532554,54.180316,1972.645170,NaN,NaN,NaN,NaN
min,1.000000,NaN,36.000000,1.000000,0.000000,NaN,NaN,NaN,NaN
25%,326.250000,NaN,71.000000,5.000000,89.250000,NaN,NaN,NaN,NaN
50%,651.500000,NaN,165.000000,10.000000,315.500000,NaN,NaN,NaN,NaN
75%,976.750000,NaN,222.500000,16.000000,818.000000,NaN,NaN,NaN,NaN


In [6]:
# Clean column names: remove spaces and standardize casing
df.columns = df.columns.str.strip().str.title()
print("Available columns:", df.columns.tolist())

# Plot histogram for HP (now 'Hp' after standardization)
if 'Hp' in df.columns:
    df['Hp'].hist(bins=20)
    plt.title('Distribution of HP')
    plt.xlabel('HP')
    plt.ylabel('Frequency')
    plt.show()
else:
    print("Column 'Hp' not found after cleanup.")


Available columns: ['Id', 'Name', 'Base_Experience', 'Height', 'Weight', 'Types', 'Abilities', 'Moves', 'Stats']
Column 'Hp' not found after cleanup.


In [9]:
# Step 1: Clean column names (remove spaces, title case for consistency)
df.columns = df.columns.str.strip().str.title()
print("Cleaned Columns:", df.columns.tolist())

# Step 2: Define possible stat columns
expected_stats = ['Hp', 'Attack', 'Defense', 'Speed']

# Step 3: Keep only the ones that actually exist in the DataFrame
available_stats = [col for col in expected_stats if col in df.columns]

# Step 4: Plot if we have at least two stats
if len(available_stats) >= 2:
    sns.pairplot(df[available_stats])
    plt.suptitle('Pairplot of Available Stats', y=1.02)
    plt.show()
else:
    print("Not enough valid numeric columns found for pairplot.")


Cleaned Columns: ['Id', 'Name', 'Base_Experience', 'Height', 'Weight', 'Types', 'Abilities', 'Moves', 'Stats']
Not enough valid numeric columns found for pairplot.


In [12]:
# Function to parse non-standard Stats string into a dictionary
def parse_stats(stats_str):
    stats_dict = {}
    for item in stats_str.split(','):
        if '=' in item:
            key, value = item.strip().split('=')
            key = key.replace('-', '_').strip().lower()   # Fix keys like "special-attack"
            stats_dict[key] = int(value.strip())
    return stats_dict

# Apply the function to the 'Stats' column
df['Stats'] = df['Stats'].apply(parse_stats)

# Extract relevant stats
df['attack'] = df['Stats'].apply(lambda x: x.get('attack', None))
df['defense'] = df['Stats'].apply(lambda x: x.get('defense', None))

# Plot using Plotly
import plotly.express as px
fig = px.scatter(
    df,
    x="attack",
    y="defense",
    color="Types",        # Assuming 'Types' column exists
    hover_name="Name",
    title="Attack vs Defense"
)
fig.show()
